<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Bayesian Statistics With Baseball Data: Priors and Posteriors

_Authors: Kiefer Katovich (SF)_

---

Let’s review the general set up for Bayesian modeling. We have:

### $$P\left(\;model\;|\;data\;\right) = \frac{P\left(\;data\;|\;model\;\right)}{P(\;data\;)} P\left(\;model\;\right)$$

This is the diachronic interpretation of Bayes' theorem.

It can also be written as:

### $$posterior \propto likelihood \cdot prior$$

Here, the posterior is an update of our prior belief given the data observed.

## Bayesian Analysis of Batting Averages

---

Load in the simple batting average data for players below. There are just four fields in the data set: the player's name, their times at bat, their hits, and their batting average.

> *Note: [This section of the lab is a partial replication of this exercise in R, if you're interested](https://www.r-bloggers.com/understanding-empirical-bayes-estimation-using-baseball-statistics/). Here, we took out the empirical Bayes section because it's not technically a correct interpretation of Bayesian statistical analysis.*

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seas

from scipy.stats import beta, binom
from scipy.stats import binom

%matplotlib inline

ModuleNotFoundError: No module named 'seas'

In [ ]:
# A:

df = pd.read_csv('./career_hits.csv')
df.head()

### 1) Find the top and bottom three hitters according to their average.

What is wrong with using the average to find the three best and worst hitters?

In [ ]:
# A:
print('The following players have the highest batting average at 1.0: ')
df[df.average==1].name

In [ ]:
print('The following players have the lowest batting average at 0.0: ')
df[df.average==0].name

The issue with using the highest and lowest batting averages are that this represents a large group of players with very few at bats who have an average of either 0 or 1.

### 2) Set up a prior distribution to represent our belief about batting average.

[After doing a quick search online](https://www.google.com/search?q=average+batting+average+for+players+mlb&oq=average+batting+average+for+players+mlb&aqs=chrome..69i57j0.7373j0j4&sourceid=chrome&ie=UTF-8), it looks like the overall batting average for MLB baseball players is around 0.260.

**Let's make it simple and say that our prior belief is that, out of 100 at-bats, we’ve seen 26 hits. Set up a beta distribution with `alpha=27` and `beta=75`. Plot it with the function from above.**

This is our distribution of beliefs on the batting average (probability of hitting while at bat) for MLB players.

In [ ]:
# A:
a_prior = 27
b_prior = 75
prior_distn = beta(a_prior, b_prior)

percentages_for_plotting_x_axis = np.linspace(0, 1, 10001)

In [ ]:
plt.plot(percentages_for_plotting_x_axis, prior_distn.pdf(percentages_for_plotting_x_axis))
plt.xlabel('Batting Average', fontsize=16)
plt.ylabel('Likelihood', fontsize=16);

### 3) Calculate the maximum a posteriori (MAP) estimate of players' batting averages.

The maximum a posteriori (MAP) estimate is the mode of a posterior distribution for a statistical parameter.

In our case, the MAP estimate for our players' batting averages will be the mode of the posterior beta distribution we get from updating our prior distribution with their at bats and hits.

---

> *Note: For an additional overview of the math behind updating the beta distribution posterior and conjugate priors, ([check out this website](https://alexanderetz.com/2015/07/25/understanding-bayes-updating-priors-via-the-likelihood/)).*

We need to update our beta distribution’s prior belief about batting averages with a player's at-bat and hit information to get a _new_ beta posterior distribution for that player's batting average. 

Luckily, with the beta distribution, the update is just a matter of adding in our new observations to the alpha and beta parameters, where alpha is the number of hits and beta is the number of misses/strikes:

    observed_hits = n_hits
    observed_misses = n_misses
    beta_prior = Beta(prior_hits+1, prior_misses+1)
    beta_posterior = Beta(prior_hits + n_nits + 1, prior_misses + n_misses + 1)
    
> **Note:** This process will also be useful in the context of A/B testing, which we’ll look at later on.

For each player, update the prior to the posterior distribution and calculate the mode of the distribution. The mode of a beta distribution is conveniently defined as:

### $$ \frac{\alpha - 1}{\alpha + \beta -2} $$

Which means we don't even need to use SciPy's beta distribution function. Just calculate the new alpha (hits) and beta (misses) for each player's posterior beta distribution and plug them into the formula above to get the MAP estimate of batting the average.

In [ ]:
df.columns

In [ ]:
# A:

df['MAP'] = (a_prior + df['hits'] - 1)/(a_prior + b_prior + df['at_bat'] - 2)
df.head()

### 4) Look up the top and bottom batters according to the MAP estimate.

In [ ]:
# A:

print('Top 3 players by MAP: ')
df.sort_values('MAP').tail(3).name

In [ ]:
print('Bottom 3 players by MAP: ')
df.sort_values('MAP').head(3).name

### 5) Plot the MLE batting average against the MAP batting average.

In [ ]:
# A:
sns.jointplot(
    x=df.average, 
    y=df.MAP, 
    kind='scatter', 
    color='b', 
    height=10
).set_axis_labels('MLE batting average', 'MAP batting average')
plt.title(
    'Maximum Posterior Batting Average versus MLE Batting Averages \n Utilizing a Prior of alpha = 27, beta = 75',
    horizontalalignment='right',
    pad = 150,
    loc = 'right'
)